# 1. 使用CrossEncoder重排

Cross-Encoder，交叉编码器。主要用于文本对间的精细相关性评估，强调的是两个文本之间的交互信息。

- Cross-Encoder 
    - 输入：\[sentence1, sentence2\]
    - 输出：句子对关系（分数或标签）
- 模型：BAAI/bge-reranker-large
- bge: BAAI General Embedding, BAAI通用嵌入

- BAAI：Beijing Academy of Artificial Intelligence, 北京智源研究院。
    一个非营利新型研究机构，致力于成为人工智能领域的创新领导者。

```python
from sentence_transformers import CrossEncoder
```

CrossEncoder的实例方法:
- predict(): 接收一个列表，列表元素为句子对（字符串对/字符串元组），返回相关度得分，得分越高说明越相关。


In [2]:
import chromadb
client = chromadb.HttpClient(host="localhost", port=8000)
collection = client.get_or_create_collection(name="rag-video-collection")
doc_info = collection.get(offset=0, limit=5)
print(doc_info["documents"])

['科技行业 2025 年展望:AI 浪潮重新 定义全球科技的未来\n\n发表时间：2024 年 12 月 13 日\n作者：\n(1) 沈岱 - 首席科技分析师 - sia_huang@spdbi.com - (852) 2809 0355\n(2) 马智焱 - 科技分析师 - ivy_ma@spdbi.com - (852) 2809 0300\n(3) 黄佳琦 - 科技分析师 - sia_huang@spdbi.com  - (852) 2809 0355\n\n一、全球 AI 行业具备较大成长空间', '2022 年 11 月底，OpenAI 发布 ChatGPT 3.5。紧随其后，生成式 AI，在供应 端的推动下，进入爆发式发展期。无论是OpenAI 随后发布的 ChatGPT 4.0， 还是谷歌、Meta 等多家厂商发布的 Gemini、Llama 等 AI 大模型，都是这一 轮AI 行业爆发式发展的体现。AI 算力芯片作为本轮 AI 浪潮基本且不可或缺 的硬件基础，成为这波浪潮最先受益的行业。', '首先，美国的云服务提供商大幅增加对于 AI 算力芯片的资本开支，并且推 动其AI 大模型快速更新迭代。而且，大量的公司不愿意在这波浪潮中落后，B 端需求初见端倪并快速成长，希望借助生成式 AI，提高生产力。\n这四家美国头部云服务厂商(微软、谷歌、亚马逊、Meta)资本支出同比增 速在经历了10 个季度的下行后，在 2023 年二季度见底触及-9%的低点，随 后就开启上行，在今年一季度实现+30%增长，并在二、三季度持续上扬至58%、59%(图表 1)。本轮美国互联网大厂资本开支投向主要集中在 AI 算 力的部署。', '展望 2025 年，我们预期美国互联网厂商以及其他初创企业都在加速部署 AI大模型算力，以避免在新技术商业落地过程中落后于竞争对手。所以，我们 预期这些美国头部科技企业的资本开支上行动能会推动AI 算力芯片需求大 幅上扬。\n其次，承接云服务厂商资本开支的 AI 服务器正在快速增长。与科技新兴产 业发展趋势类似，目前AI 服务器渗透率也处于加速上扬阶段，速度快于一 年/半年以前的市场预测。根据 Counterpoint，AI 服务器收入渗透率已经在二 季度达到29%(图表 8)。', 'TrendForce 预计 AI 服务器出货量将从 

In [3]:
# BAAI/bge-reranker-large
from sentence_transformers import CrossEncoder
model = CrossEncoder("BAAI/bge-reranker-large")
# todo
question = "2024年中东非智能手机的出货量为多少？"
scores = model.predict([(question, doc_item) for doc_item in doc_info["documents"]])
print(scores)

/opt/miniconda3/envs/huggingface/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.00019811 0.00026394 0.00077612 0.00020138 0.0022332 ]


In [10]:
# list1, list2
# zip: [(list1_0, list2_0), (list1_1, list2_1)]
# for s in zip(scores, doc_info["documents"]):
#     print(s)
sorted(zip(scores, doc_info["documents"]), reverse=True, key=lambda x:x[0])

[(0.002233205,
  'TrendForce 预计 AI 服务器出货量将从 2023 年的 118 万台增长至 2024 年的167 万台，预计 2025 年将会增长至 214 万台。根据 TrendForce 预测，2023年、2024 年、2025 年，全球 AI 服务器出货量渗透率达到/预计达到 8.8%、12.2%、15.0%(图表 7)。'),
 (0.0007761158,
  '首先，美国的云服务提供商大幅增加对于 AI 算力芯片的资本开支，并且推 动其AI 大模型快速更新迭代。而且，大量的公司不愿意在这波浪潮中落后，B 端需求初见端倪并快速成长，希望借助生成式 AI，提高生产力。\n这四家美国头部云服务厂商(微软、谷歌、亚马逊、Meta)资本支出同比增 速在经历了10 个季度的下行后，在 2023 年二季度见底触及-9%的低点，随 后就开启上行，在今年一季度实现+30%增长，并在二、三季度持续上扬至58%、59%(图表 1)。本轮美国互联网大厂资本开支投向主要集中在 AI 算 力的部署。'),
 (0.00026394174,
  '2022 年 11 月底，OpenAI 发布 ChatGPT 3.5。紧随其后，生成式 AI，在供应 端的推动下，进入爆发式发展期。无论是OpenAI 随后发布的 ChatGPT 4.0， 还是谷歌、Meta 等多家厂商发布的 Gemini、Llama 等 AI 大模型，都是这一 轮AI 行业爆发式发展的体现。AI 算力芯片作为本轮 AI 浪潮基本且不可或缺 的硬件基础，成为这波浪潮最先受益的行业。'),
 (0.00020137997,
  '展望 2025 年，我们预期美国互联网厂商以及其他初创企业都在加速部署 AI大模型算力，以避免在新技术商业落地过程中落后于竞争对手。所以，我们 预期这些美国头部科技企业的资本开支上行动能会推动AI 算力芯片需求大 幅上扬。\n其次，承接云服务厂商资本开支的 AI 服务器正在快速增长。与科技新兴产 业发展趋势类似，目前AI 服务器渗透率也处于加速上扬阶段，速度快于一 年/半年以前的市场预测。根据 Counterpoint，AI 服务器收入渗透率已经在二 季度达到29%(图表 8)。'),
 (0.00019811449,
  '科技行业 2025 年展望:AI 浪潮重新

In [1]:
# 1. 添加文档到chromadb当中
# 2. 根据问题进行检索并输出答案

import chromadb
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import CrossEncoder

def get_metadata_str(metadata):
    if metadata is None:
        return ""
    _str = ""
    # (key, value)
    for key, value in metadata.items():
        _str += f"{key}:{value}\n"
    return _str

class RagLangChain():
    def __init__(self, 
                 collection_name: str = "rag-video-collection", 
                 host: str = "localhost",
                port: int = 8000):
        self.collection_name = collection_name
        self.host = host
        self.port = port

    def __get_vector_store(self):
        chromadb_client = chromadb.HttpClient(host=self.host, port=self.port)
        embedding_fn = OllamaEmbeddings(model="nomic-embed-text:latest")
        
        vector_store = Chroma(collection_name=self.collection_name,
                              client=chromadb_client, 
                              embedding_function=embedding_fn
                             )
        return vector_store

    def delete_collection(self):
        vector_store = self.__get_vector_store()
        vector_store.delete_collection()
        
    def add_file(self, file_path: str, metadata = None):
        loader = TextLoader(file_path)
        
        docs = loader.load()
        text_spliter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
        all_splits = text_spliter.split_documents(docs)

        if metadata is not None:
            for split_item in all_splits:
                split_metadata = split_item.metadata
                split_item.metadata = {**split_metadata, **metadata}
        
        vector_store = self.__get_vector_store()
        ids = vector_store.add_documents(documents=all_splits)
        return ids

    def __query_vector(self, info):
        vector_store = self.__get_vector_store()
        retriever = vector_store.as_retriever(
            # search_type="mmr",
            # search_kwargs={"k": 16, "fetch_k": 20, "lambda_mult": 0.1}
            search_kwargs={"k": 20}
        )

        # Document对象,id/page_content/metadata
        docs = retriever.invoke(info["query"])

        model = CrossEncoder("BAAI/bge-reranker-large")
        # question = "2024年中东非智能手机的出货量为多少？"
        scores = model.predict([(info["query"], doc_item.page_content) for doc_item in docs])
        # [(score, Document), (score, Document), ...]
        sorted_list = sorted(zip(scores, docs), reverse=True, key=lambda x:x[0])
        docs = [doc for _, doc in sorted_list]
        docs = docs[:4]
        score_list = [score for score, _ in sorted_list]
        # (0, item0), (1, item1)
        for index, doc_item in enumerate(docs):
            print(f"文档片段{index}")
            print(f"CrossEncoder得分:{score_list[index]}")
            print(doc_item)
            print("*" * 80)
        # docs_str = "\n\n".join(doc.page_content for doc in docs)
        docs_str = "\n\n".join(f"""
{get_metadata_str(doc.metadata)}
{doc.page_content}""" for doc in docs)
        # print(f"@@@@@{docs_str}")
        return docs_str
        
    def query(self, question: str):
        prompt = ChatPromptTemplate.from_template("""
你是一个问答机器人。你的任务是根据下述给定的已知信息回答用户问题。

已知信息:
{context}
用户问题：
{query}
如果已知信息不包含用户问题的答案，或者已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。
请不要输出已知信息中不包含的信息或答案。
请用中文回答用户问题。
""")
        
        llm = ChatOllama(model="qwen2.5:latest")
        
        output_parser = StrOutputParser()
        # 返回值: {"query": "XXXX", "context": "XXXXX2"}
        chain = ( {"context": self.__query_vector, "query": lambda x: x["query"]} | prompt | llm | output_parser)
        
        result = chain.invoke({"query": question})
        return result


/opt/miniconda3/envs/rag-video/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rag_chain = RagLangChain()

In [3]:
rag_chain.query(question="2024年中东非智能手机的出货量为多少？")

文档片段0
CrossEncoder得分:0.9604961276054382
page_content='从地区来看，中东非的出货量为 4,199 万部，同比增速为 14%，表现亮眼， 唯一一个达到双位数增速的地区。紧随其后的亚洲其他地区出货量4,502 万 部，同比增长8%。这两个区域的出货量表现好于我们此前预测。虽然印度 三季度的出货量同比增长5%，但是略微低于我们此前预测。
在三季度，中国的出货量 6,878 万部，同比增长 3%，环比下滑 4%(图表32)。欧洲和北美发达市场的出货量分别同比下降 5%和 1%。这三个区域的 出货量表现大体符合我们此前的预测。' metadata={'source': '科技行业 2025 年展望.txt', '作者': '沈岱,马智焱,黄佳琦', '发表时间': '2024 年 12 月 13 日', '文章标题': '科技行业2025年展望'}
********************************************************************************
文档片段1
CrossEncoder得分:0.22231201827526093
page_content='三、智能手机:2024 年、2025 年连续两年保持增长
预计 2025 年全球智能手机出货量将同比增长 2%
根据 IDC 数据，今年三季度全球智能手机出货量 3.15 亿部，环比增长 8%， 同比增长4%(图表 34)。虽然同比增速较一二季度下行，但是同比增速略 好于我们此前预测。这与我们与智能手机供应链沟通下来的情况类似，比半 年报看到的行业需求情况略好。' metadata={'source': '科技行业 2025 年展望.txt', '作者': '沈岱,马智焱,黄佳琦', '发表时间': '2024 年 12 月 13 日', '文章标题': '科技行业2025年展望'}
********************************************************************************
文档片段2
CrossEncoder得分:0.17537850141525269
page_content='高端机型相对优异的表现与我们在安卓品牌端以及供应链端看到的情况

'2024年中东非智能手机的出货量为4,199万部。'